In [20]:
import numpy as np
import sys
import torch
import random
import robosuite as suite
from robosuite.environments.manipulation.empty import Empty
from scipy import interpolate
from robosuite.utils.mjmod import DynamicsModder
import xml.etree.ElementTree as ET

In [2]:
horizon = 200

In [3]:
def plan(start_pose, middle_pose, end_pose, horizon):
    cs = interpolate.CubicSpline([0, 2 * horizon // 3, 5*horizon // 6, horizon], [start_pose, middle_pose, end_pose, end_pose], 
                                       axis=0, bc_type='clamped')
    return cs(range(horizon))  
def difference(traj):
    traj = np.array(traj)
    new_traj = [traj[0]]
    for i in range(len(traj)):
        if i == 0:
            continue
        new_traj.append(traj[i] - traj[i - 1])
    return new_traj

In [4]:
suite.environments.base.register_env(Empty)

In [5]:
controller_config = suite.load_controller_config(default_controller="JOINT_POSITION")

In [6]:
env = suite.make(
    "Empty",
    robots="IIWA",             # load a Sawyer robot and a Panda robot
    gripper_types="ClothGripper",                # use default grippers per robot arm
    controller_configs=controller_config, 
    has_renderer=True,                      # on-screen rendering
    render_camera="sideview",              # visualize the "frontview" camera
    has_offscreen_renderer=False,           # no off-screen rendering
    render_collision_mesh=True,
    control_freq=20,                        # 20 hz control for applied actions
    horizon=horizon,                            # each episode terminates after 200 steps
    use_object_obs=False,                   # no observations needed
    use_camera_obs=False,                   # no observations needed
)

Creating window glfw


In [155]:
def modify_xml_return_params():
    # TODO: make random values instead of fixed
    springdamper_time_constant = 0.002
    springdamper_damping_ratio = 2
    frictionloss = 3
    tendon_stiffness = 1 # random.uniform(0, 10)
    tendon_damping = 1 # random.uniform(0, 10)
    tendon_frictionloss = 3 # random.uniform(0, 10)
    
    # TODO: convert to string
    springdamper_str = str(springdamper_time_constant) + ' ' + str(springdamper_damping_ratio)
    frictionloss_str = str(frictionloss)
    tendon_stiffness_str = str(tendon_stiffness)
    tendon_damping_str = str(tendon_damping)
    tendon_frictionloss_str = str(tendon_frictionloss)
    
    tree = ET.parse('./robosuite/robosuite/models/assets/grippers/cloth.xml')
    root = tree.getroot()
    print('THIS IS ROOT: ', root)
    
    cloth_joint = root.find('default').find('default').find('joint')
    print('THIS IS CLOTH JOINT: ', cloth_joint)
    cloth_joint.set('springdamper', springdamper_str)
    cloth_joint.set('frictionloss', frictionloss_str)
    
    tendon = root.find('tendon')
    for t in tendon.iter('spatial'):
        t.set('stiffness', tendon_stiffness_str)
        t.set('damping', tendon_damping_str)
        t.set('frictionloss', tendon_frictionloss_str)
        
    tree.write('./robosuite/robosuite/models/assets/grippers/cloth.xml')
        
    return springdamper_time_constant, springdamper_damping_ratio, frictionloss, tendon_stiffness, tendon_damping, tendon_frictionloss
    

In [156]:
import tqdm
import time

num_dataset_size = 1000
num_cloth_joints = 10

all_geom_positions = []
all_parameters = []
for _ in tqdm.tqdm(range(num_dataset_size)):
    done = False
    obs = env.reset()
    
    params = modify_xml_return_params()
    print('params: ', params)
    all_parameters.append(params)
    
#     for i in range(num_cloth_joints):
#         modder.mod("gripper0_joint" + str(i), "damping", 3)
#         modder.mod("gripper0_joint" + str(i), "stiffness", 0)

    
    geom_positions = []
#     traj = difference(plan([0, 0, 0, 0, 0, 0, 0], [0, -3, 0, 3, 0, -3, 0], [0, 0, 0, 0, 0, 0, 0], horizon))
    
    initial_jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) for joint in range(1, 8)])
    final_jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) + (1.2 if joint == 2 else 0) for joint in range(1, 8)])
    
    i = 0
    robot_poses = []
    while not done:
        robot_poses.append([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) for joint in range(1, 8)])
#         action = traj[i]
        i += 1
        jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) for joint in range(1, 8)])
        relative_jpos = final_jpos - jpos
        
#         for j in range(num_cloth_joints):
#             pos = env.sim.data.geom_xpos[env.sim.model.geom_name2id("gripper0_geom{}".format(j))]
#             geom_positions.append(pos[0])
#             geom_positions.append(pos[2])
            
        obs, reward, done, info = env.step(relative_jpos) 
        env.render()
#     with open("robot_poses.txt", 'w') as f:
#         for pose in robot_poses:
#             print(pose)
#             f.write("[{}],\n".format(", ".join([str(p) for p in pose])))
#     sys.exit()
    all_geom_positions.append(geom_positions)
    
all_geom_positions = np.array(all_geom_positions)
all_parameters = np.array(all_parameters)

env.close()

  0%|          | 0/1000 [00:01<?, ?it/s]

Creating window glfw
THIS IS ROOT:  <Element 'mujoco' at 0x7fb2b0ef89a8>
THIS IS CLOTH JOINT:  <Element 'joint' at 0x7fb2b264e278>


AttributeError: 'NoneType' object has no attribute 'iter'

In [132]:
loss_fn = torch.nn.GaussianNLLLoss()
for epoch in range(100):
    model.train()
    train_losses = []
    for position, target_params in train_loader:
        position = position.to("cuda").float()
        target_params = target_params.to("cuda").float()
        
        pred_params_mu, pred_params_logvar = torch.split(model(position), 2, dim=1)
        loss = loss_fn(pred_params_mu, target_params, torch.exp(pred_params_logvar))
        
        train_losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_losses = []
        for position, target_params in val_loader:
            position = position.to("cuda").float()
            target_params = target_params.to("cuda").float()

            pred_params_mu, pred_params_logvar = torch.split(model(position), 2, dim=1)
            loss = loss_fn(pred_params_mu, target_params, torch.exp(pred_params_logvar))

            val_losses.append(loss.item())
        
    print("Epoch: {}, Train Loss: {}, Val Loss: {}".format(epoch, np.mean(train_losses), np.mean(val_losses)))

NameError: name 'model' is not defined

In [ ]:
predictions = []
ground_truth = []
for i, (position, target_params) in enumerate(val_loader):
    prediction, _ = torch.split(model(position.to("cuda").float()), 2, dim=1)
    predictions.append(prediction.detach().cpu().numpy())
    ground_truth.append(target_params.cpu().numpy())
predictions = np.concatenate(predictions, axis=0)
ground_truth = np.concatenate(ground_truth, axis=0)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(ground_truth[:, 0], predictions[:, 0])
plt.show()

In [ ]:
from matplotlib import pyplot horizonas plt
plt.scatter(ground_truth[:, 1], predictions[:, 1])
plt.show()

In [ ]:
# <spatial name="tendon1" stiffness="100000" damping="0" >
# 			<site site="anchor_left"/> 
# 			<geom geom="geom0" sidesite="gripper0_site0_left"/>
# 			<site site="site1_left"/>
# 			<geom geom="geom2" sidesite="gripper0_site2_left"/>
# 			<site site="site3_left"/>
# 			<geom geom="geom4" sidesite="gripper0_site4_left"/>
# 			<site site="site5_left"/>
# 			<geom geom="geom6" sidesite="gripper0_site6_left"/>
# 			<site site="site7_left"/>
# 			<geom geom="geom8" sidesite="gripper0_site8_left"/>
# 			<site site="site9_left"/>
# 			<geom geom="geom10" sidesite="gripper0_site10_left"/>
# 			<site site="site11_left"/>
# 		</spatial>
# 		<spatial name="tendon2" stiffness="100000" damping="0">
# 			<site site="anchor_left"/>
# 			<geom geom="geom1" sidesite="gripper0_site1_left"/>
# 			<site site="site2_left"/>
# 			<geom geom="geom3" sidesite="gripper0_site3_left"/>
# 			<site site="site4_left"/>
# 			<geom geom="geom5" sidesite="gripper0_site5_left"/>
# 			<site site="site6_left"/>
# 			<geom geom="geom7" sidesite="gripper0_site7_left"/>
# 			<site site="site8_left"/>
# 			<geom geom="geom9" sidesite="gripper0_site9_left"/>
# 			<site site="site10_left"/>
# 		</spatial>
# 		<spatial name="tendon3" stiffness="100000" damping="0" >
# 			<site site="anchor_right"/> 
# 			<geom geom="geom0" sidesite="gripper0_site0_right"/>
# 			<site site="site1_right"/>
# 			<geom geom="geom2" sidesite="gripper0_site2_right"/>
# 			<site site="site3_right"/>
# 			<geom geom="geom4" sidesite="gripper0_site4_right"/>
# 			<site site="site5_right"/>
# 			<geom geom="geom6" sidesite="gripper0_site6_right"/>
# 			<site site="site7_right"/>
# 			<geom geom="geom8" sidesite="gripper0_site8_right"/>
# 			<site site="site9_right"/>
# 			<geom geom="geom10" sidesite="gripper0_site10_right"/>
# 			<site site="site11_right"/>
# 		</spatial>
# 		<spatial name="tendon4" stiffness="100000" damping="0" >
# 			<site site="anchor_right"/>
# 			<geom geom="geom1" sidesite="gripper0_site1_right"/>
# 			<site site="site2_right"/>
# 			<geom geom="geom3" sidesite="gripper0_site3_right"/>
# 			<site site="site4_right"/>
# 			<geom geom="geom5" sidesite="gripper0_site5_right"/>
# 			<site site="site6_right"/>
# 			<geom geom="geom7" sidesite="gripper0_site7_right"/>
# 			<site site="site8_right"/>
# 			<geom geom="geom9" sidesite="gripper0_site9_right"/>
# 			<site site="site10_right"/>
# 		</spatial>

In [ ]:
# Old large
# 0.2 - 2.7 cm
# 3.9 - 6.3 cm
# 7.7 - 10.2 cm
# 11.5 - 14 cm
# 15.4 - 17.8 cm
# 18.2 - 20.7 cm

# New small
# 0.3 - 2.3 cm
# 4.5 - 6.5 cm
# 8.5 - 10.5 cm
# 12.6 - 14.6 cm
# 16.7 - 18.7 cm
# 20.7 x 27.5 cm